In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('test') \
                    .getOrCreate()

23/08/21 13:55:56 WARN Utils: Your hostname, codespaces-d00206 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/08/21 13:55:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 13:55:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Definitionally, a DataFrame consists of a series of records (like rows in a table), that are of type Row, and a number of columns (like columns in a spreadsheet) that represent a computation
expression that can be performed on each individual record in the Dataset. Schemas define the name as well as the type of data in each column. Partitioning of the DataFrame defines the layout of the DataFrame or Dataset’s physical distribution across the cluster. The partitioning scheme defines how that is allocated. You can set this to be based on values in a certain column or nondeterministically.

Let’s create a DataFrame with which we can work:

In [6]:
df = spark.read.format('json').load('../data/flight-data/json/2015-summary.json')

We discussed that a DataFame will have columns, and we use a schema to define them. Let’s
take a look at the schema on our current DataFrame:

In [7]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



# Schemas

Let’s begin with a simple file and let the semi-structured nature of line-delimited JSON define the structure:

In [8]:
spark.read.format('json').load('../data/flight-data/json/2015-summary.json').schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

If the types in the data (at runtime) do not match the schema, Spark will throw an error. The example that follows shows how to create and enforce a specific schema on a DataFrame.

In [8]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
    StructField('DEST_COUNTRY_NAME', StringType(), True),
    StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
    StructField('count', LongType(), False, metadata={'hello':'world'}),
])

df = spark.read.format('json').schema(myManualSchema).load('../data/flight-data/json/2015-summary.json')

# Columns and Expressions

## Columns

There are a lot of different ways to construct and refer to columns but the two simplest ways are by using the col or column functions. To use either of these functions, you pass in a column name:

In [9]:
from pyspark.sql.functions import col, column

col('someColumnName')
column('someColumnName2')

Column<'someColumnName2'>

If you need to refer to a specific DataFrame’s column, you can use the col method on the specific DataFrame (Scala). This can be useful when you are performing a join and need to refer to a specific column in one DataFrame that might share a name with another column in the joined DataFrame. As an added benefit, Spark does not need to resolve this column itself (during the analyzer phase) because we did that for Spark:


In [12]:
df['count']

Column<'count'>

In [14]:
df.DEST_COUNTRY_NAME

Column<'DEST_COUNTRY_NAME'>

## Expressions

We mentioned earlier that columns are expressions, but what is an expression? An expression is a set of transformations on one or more values in a record in a DataFrame. Think of it like a function that takes as input one or more column names, resolves them, and then potentially applies more expressions to create a single value for each record in the dataset. Importantly, this “single value” can actually be a complex type like a Map or Array.

In the simplest case, expr("someCol") is equivalent to col("someCol").

Sometimes, you’ll need to see a DataFrame’s columns, which you can do by using something like printSchema; however, if you want to programmatically access columns, you can use the columns property to see all columns on a DataFrame:

In [15]:
spark.read.format('json').load('../data/flight-data/json/2015-summary.json').columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

# Records and Rows